In [ ]:
import zipfile, struct, warnings, requests, struct
from io import StringIO, BytesIO

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import astropy
from astropy.io.votable import parse_single_table
from astroquery.utils.tap.core import TapPlus
import pvl

from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode()

from IPython.display import Image, display

%matplotlib inline
# warnings.simplefilter('ignore', category=astropy.io.votable.exceptions.VOTableSpecWarning)
# astropy.io.votable.tree
# warnings.simplefilter('ignore', category=skimage.util.dtype.warn)
warnings.simplefilter('ignore')

In [ ]:
tap_url = 'https://archives.esac.esa.int/psa/epn-tap/tap/'
psa = TapPlus(url=tap_url)

In [ ]:
query = 'SELECT COUNT(1) from epn_core'
result = psa.launch_job(query).get_data()
result['count'][0]

In [ ]:
query = 'SELECT SUM(access_estsize) from epn_core' 
result = psa.launch_job(query).get_data() # in kbyte
result['sum'][0]/1024./1024./1024.

In [ ]:
query = 'SELECT TOP 10 granule_uid, access_estsize from epn_core WHERE access_estsize IS NOT NULL ORDER BY access_estsize DESC'
result = psa.launch_job(query).get_data()
result

## MIDAS specific filtering of products

Each MIDAS product is described by a three-letter acronym at the start. You can read more about the archiving conventions here: http://pdssbn.astro.umd.edu/holdings/ro-c-midas-3-esc2-samples-v1.0/document/mid_eaicd.pdf. If you want to get really technical you can also look at the User Manual [http://pdssbn.astro.umd.edu/holdings/ro-c-midas-3-esc2-samples-v1.0/document/mid_user.pdf].

MIDAS images all start with IMG and the two letters at the end of the product name describe the image channels used - the most useful is ZS, which is the topographic channel. So we'll add some new colums to allow us to select the data type and channel, and filter to just those:

In [ ]:
query = "SELECT * FROM epn_core WHERE target_name='67P' AND instrument_name='MIDAS'"
data = psa.launch_job(query).get_data().to_pandas()

# tidy it up a little
str_df = data.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for col in str_df:
    data[col] = str_df[col]
    
len(data)